# Two Dimensional Schrödinger Equation

We would like to solve the Schrödinger Equation (SE) in two dimensions. The SE describes the time evolution of a wavefunction, and is in natural units given by
$$
i\frac{\partial \Psi(\pmb{r},t)}{\partial t} = - \nabla^2 \Psi(\pmb{r},t) + V(\pmb{r})\Psi(\pmb{r},t) = \hat{H}\Psi(\pmb{r},t),
$$
where the potential $V(\pmb{r})$ is a double slit potential landscape: zero everywhere except for a barrier with two holes in it. $\hat{H}$ is the hamiltonian. In this system we would like to see interference effects. Below we will use two different methods to solve the SE using numerical approximation schemes. But first we need some general settings.

## General settings

In [1]:
# The usual
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import linalg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib auto

# This is a non standard pacakge
# To install it, go to the anaconda prompt and type 'conda install h5py'
import h5py

# To use this code you need to install FFMpeg, help can be found on this website
# http://www.wikihow.com/Install-FFmpeg-on-Windows
plt.rcParams['animation.ffmpeg_path'] = "C:/ffmpeg/bin/ffmpeg.exe"
FFwriter = animation.FFMpegWriter()

Using matplotlib backend: Qt4Agg


In [44]:
Nx = 500
Ny = 1000
#Make real-space grid
x = np.linspace(-5, 5, Nx)
dx = 10 / Nx
y = np.linspace(-10, 10, Ny)
dy = 20 / Ny
X, Y = np.meshgrid(x, y)


#Create potentiallandscape
V_mat = np.zeros((Nx,Ny))
V_mat[0:225, 495:505] = 1000000
V_mat[235:265, 495:505,] = 1000000
V_mat[275:500, 495:505,] = 1000000

## Crank-Nicolson Method

In the Crank-Nicolson method the time evolution of the wave function is approximated using:
$$
\Psi(\pmb{r},t+\Delta t) = e^{-i \hat{H} \Delta t} \Psi(\pmb{r},t) \approx \frac{1-i\frac{1}{2}\hat{H}\Delta t}{1+i\frac{1}{2}\hat{H}\Delta t} \Psi(\pmb{r},t)
$$
which means that for every timestep we should solve the linear system
$$
(1+i\frac{1}{2}\hat{H}\Delta t) \Psi(\pmb{r},t+\Delta t) = (1-i\frac{1}{2}\hat{H}\Delta t)\Psi(\pmb{r},t)
$$
The biconjugate gradient stabalized method is used for this purpose.
The momentum operator is discretized using finite difference methods.

In [86]:
dt = 0.005
Nt = 50

# Allow k values on discrete grid
k_four = 2 * np.pi * np.fft.fftfreq(Ny, dy)


def multvec(psi_in):  
    """Generalized lineair operator which performs multiplication of wave function psi_in 
    with 1 + i dt H /2"""
    psi = np.reshape(psi_in,(Nx,Ny))
    out = psi + 1j*dt*( dx**-2 * (4*psi - np.roll(psi,1,axis=0)-np.roll(psi,-1,axis=0)\
                        -np.roll(psi,1,axis=1)-np.roll(psi,-1,axis=1))\
                        + V_mat*psi )/2
    return np.reshape(out,np.shape(psi_in))

A = linalg.LinearOperator(dtype = complex, shape = (Nx*Ny,Nx*Ny) , matvec = multvec )

In [87]:
# Create file
f = h5py.File('.\Data\TwoD\CrankNicolson37.hdf5')
# Create dataset: dataQM
datax = f.create_dataset('dataqm', (Nx,Ny,1), dtype = np.complex64, maxshape = (Nx,Ny,None))

# Define arrays
psi_x = np.zeros((Nx, Ny), dtype = complex)
psi_k = np.zeros((Nx, Ny), dtype = complex)
b     = np.zeros((Nx, Ny), dtype = complex)

width = 5
y0 = -7
k0 = 5
#  Create gaussian wave packet with width_k = 1/width
psi_k[0,:] = ((width / np.pi)**(1/4)
            * np.exp(-0.5 * (width * (k_four - k0)) ** 2 - 1j * (k_four - k0) * y0))

psi_x = np.fft.fftshift(np.fft.ifft2(psi_k))

datax[:,:,0] = psi_x
f.flush()

for i in range(Nt-1):
    # Find new wavefunction using Crank-Nicolson algorithm
    b = psi_x - 1j*dt*( dx**-2 * (4*psi_x - np.roll(psi_x,1,axis=0)-np.roll(psi_x,-1,axis=0)\
                        -np.roll(psi_x,1,axis=1)-np.roll(psi_x,-1,axis=1))\
                        + V_mat*psi_x )/2
    
    b = np.reshape(b,(Nx*Ny))
    
    #Get wavefunction at next timestep
    psi_x, succes = linalg.bicgstab(A,b, x0 = np.reshape(psi_x,(Nx*Ny)))
    psi_x = np.reshape(psi_x,(Nx,Ny))
    b = np.reshape(b,(Nx,Ny))
    
    # Save wave function to drive every interation
    print('i is', i,' and succes is ', succes == 0)
    datax.resize((Nx,Ny,datax.shape[2]+1))
    datax[:,:,-1] = psi_x
    f.flush()
        
f.close()

i is 0  and succes is  True
i is 1  and succes is  True
i is 2  and succes is  True
i is 3  and succes is  True
i is 4  and succes is  True
i is 5  and succes is  True
i is 6  and succes is  True
i is 7  and succes is  True
i is 8  and succes is  True
i is 9  and succes is  True
i is 10  and succes is  True
i is 11  and succes is  True
i is 12  and succes is  True
i is 13  and succes is  True
i is 14  and succes is  True
i is 15  and succes is  True
i is 16  and succes is  True
i is 17  and succes is  True
i is 18  and succes is  True
i is 19  and succes is  True
i is 20  and succes is  True
i is 21  and succes is  True
i is 22  and succes is  True
i is 23  and succes is  True
i is 24  and succes is  True
i is 25  and succes is  True
i is 26  and succes is  True
i is 27  and succes is  True
i is 28  and succes is  True
i is 29  and succes is  True
i is 30  and succes is  True
i is 31  and succes is  True
i is 32  and succes is  True
i is 33  and succes is  True
i is 34  and succes is  

## Split-operator Method

In the split-operator method the following time evolution operator is used:
$$
\Psi(\pmb{r},t+\Delta t) = IFFT\Big\{e^{-i \Delta t \hat{P}}FFT\Big\{e^{-i \Delta t \hat{V}} \Psi(\pmb{r},t)\Big\}\Big\}
$$
where $\hat{P}$ is the momentum operator and $\hat{V}$ is the potential operator. In the split operator scheme we do not need the momentum operator in the x-basis, it is only needed in the p-basis. For this reason we do not have to construct 3D operators, we only need the elements which would be on the diagonal. Then we can do elementwise multiplication.

In [82]:
# Split operator requires a smaller timestep
dt = 0.0001
Nt = 2500

f_four_x = 2 * np.pi * np.fft.fftfreq(Nx, dx)
f_four_y = 2 * np.pi * np.fft.fftfreq(Ny, dy)

Px, Py = np.meshgrid(f_four_x, f_four_y, indexing='ij')
exponentP = np.exp(-1j*dt*np.pi*(np.absolute(Px)**2 + np.absolute(Py)**2))
exponentV = np.exp(-1j*dt*V_mat)

In [83]:
# Create file
f = h5py.File('.\Data\TwoD\SplitOperator.hdf5')
# Create dataset: dataQM
datax = f.create_dataset('dataqm', (Nx,Ny,1), dtype = np.complex64, maxshape = (Nx,Ny,None))


# Define arrays
psi_x = np.zeros((Nx, Ny), dtype = complex)
psi_k = np.zeros((Nx, Ny), dtype = complex)

width = 5
y0 = -7
k0 = 5
#  Create gaussian wave packet with width_k = 1/width
psi_k[0,:] = ((width / np.pi)**(1/4)
            * np.exp(-0.5 * (width * (k_four - k0)) ** 2 - 1j * (k_four - k0) * y0))

psi_x = np.fft.fftshift(np.fft.ifft2(psi_k))
datax[:,:,0] = psi_x
f.flush()

for i in range(Nt-1):
    term1 = np.fft.fft2(exponentV*psi_x, norm = "ortho")
    psi_x = np.fft.ifft2(exponentP*term1, norm = "ortho")
    if i%50 == 0:
        # Save wave function to drive once every 50 interations
        print(i, end = " ")
        datax.resize((Nx,Ny,datax.shape[2]+1))
        datax[:,:,-1] = psi_x
        f.flush()
        
f.close()        

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 

## Make animation 

In [88]:
fig = plt.figure()
f = h5py.File(".\Data\TwoD\CrankNicolson37.hdf5","r")

#Initalize figure
ims = []
Nframe = f['dataqm'].shape[2]
for i in range(Nframe):
    im = plt.imshow(np.absolute(f['dataqm'][:,:,i])**2, vmin = 0, vmax = 10**-11 , cmap=plt.get_cmap('viridis'), animated=True)
    ims.append([im])

anim = animation.ArtistAnimation(fig, ims, interval = 200, blit = True, repeat_delay = 1000)
plt.colorbar()
plt.title(r'Crank Nicolson simulation of double slit')
anim.save(r'.\Animations\2DCrankNicolson.mp4',writer = FFwriter, fps = 2)

f.close()